In [10]:
import os
import requests
import pandas as pd
import time
from dotenv import load_dotenv

In [2]:
load_dotenv() 
API_KEY = os.getenv("ABUSEIPDB_API_KEY")

## Descargar la Blacklist

In [3]:
BLACKLIST_URL = "https://api.abuseipdb.com/api/v2/blacklist"

HEADERS = {
    "Accept": "application/json",
    "Key": API_KEY
}

params = {
    "confidenceMinimum": "90"
}

response = requests.get(BLACKLIST_URL, headers=HEADERS, params=params)

if response.status_code != 200:
    print("Error:", response.status_code, response.text)
    exit()



In [4]:
response = requests.get(BLACKLIST_URL, headers=HEADERS, params=params)


In [5]:
# --- Convertir respuesta JSON a Python ---
data_json = response.json()
# la clave 'data' contiene la lista de IPs con detalles
blacklist_data = data_json.get("data", [])

# --- Convertir a DataFrame para análisis o CSV ---
df = pd.DataFrame(blacklist_data)

# --- Guardar CSV ---
df.to_csv("abuseipdb_blacklist.json.csv", index=False)

In [6]:
print(f"Se descargaron {len(df)} IPs maliciosas con score >=90")
print("Primeras 5 IPs con detalle:")
print(df.head())

Se descargaron 10000 IPs maliciosas con score >=90
Primeras 5 IPs con detalle:
         ipAddress countryCode  abuseConfidenceScore  \
0   101.78.161.172          HK                   100   
1   91.244.113.178          RU                   100   
2  203.150.162.250          TH                   100   
3     80.94.95.112          RO                   100   
4   14.103.127.242          CN                   100   

              lastReportedAt  
0  2025-09-19T14:17:02+00:00  
1  2025-09-19T14:17:02+00:00  
2  2025-09-19T14:17:02+00:00  
3  2025-09-19T14:17:02+00:00  
4  2025-09-19T14:17:02+00:00  


## Consultar IP

In [7]:
CHECK_URL = "https://api.abuseipdb.com/api/v2/check"

In [8]:
def check_ips_confidence(ips, pause=1.0):
    """
    Recibe lista de IPs y devuelve un diccionario {IP: abuseConfidenceScore}
    pause: segundos a esperar entre peticiones para no saturar la API
    """
    results = {}
    for ip in ips:
        try:
            params = {"ipAddress": ip, "maxAgeInDays": "90"}
            r = requests.get(CHECK_URL, headers=HEADERS, params=params)
            r.raise_for_status()
            data = r.json().get("data", {})
            score = data.get("abuseConfidenceScore", 0)
            results[ip] = score
            print(f"{ip} -> {score}%")
        except Exception as e:
            print(f"Error con {ip}: {e}")
            results[ip] = None
        time.sleep(pause)
    return results

In [11]:
# EJEMPLO DE USO

ips = ["180.76.105.165", "143.92.37.165", "1.1.1.1"]
confidence_scores = check_ips_confidence(ips)
print(confidence_scores)

180.76.105.165 -> 100%
143.92.37.165 -> 100%
1.1.1.1 -> 0%
{'180.76.105.165': 100, '143.92.37.165': 100, '1.1.1.1': 0}


In [12]:
#¿Filtrado?

ipmaliciosas = {ip: score for ip, score in confidence_scores.items() if score >= 75}
print("IPs consideradas maliciosas:", ipmaliciosas)

IPs consideradas maliciosas: {'180.76.105.165': 100, '143.92.37.165': 100}


In [13]:
#Pasar a Dataframe

df = pd.DataFrame(list(confidence_scores.items()), columns=["IP", "ConfidenceScore"])
df.to_csv("ipmaliciosas.csv", index=False)
print("Archivo confidence_scores.csv guardado.")

Archivo confidence_scores.csv guardado.
